In [2]:
!pip install git+https://github.com/jkminder/nnpatch

Using runai image conda
  Cloning https://github.com/jkminder/nnpatch to /tmp/pip-req-build-9xpnfjzv
  Running command git clone --filter=blob:none --quiet https://github.com/jkminder/nnpatch /tmp/pip-req-build-9xpnfjzv
  Resolved https://github.com/jkminder/nnpatch to commit b36cf9e59b1c37e6ac2a346e11fc18939005a125
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nnpatch: filename=nnpatch-0.0.0-py3-none-any.whl size=18550 sha256=704ce04d223c39a9dd5ef27f72732742d0cdcd2d3aa3efbdc07389fb831287fb
  Stored in directory: /tmp/pip-ephem-wheel-cache-fea4huv6/wheels/15/af/9e/8d61a7f30517f371092b43435cd86cf8393d3316c1baf1862d
Successfully built nnpatch


In [1]:
from nnpatch.subspace import LowRankOrthogonalProjection, BinaryHook
from dlabutils import model_path
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model = AutoModelForCausalLM.from_pretrained(model_path("meta-llama/Meta-Llama-3.1-8B-Instruct"), torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path("meta-llama/Meta-Llama-3.1-8B-Instruct"))
tokenizer.pad_token = None

proj = LowRankOrthogonalProjection.from_pretrained("jkminder/CTXPRIOR-Projection-Meta-Llama-3.1-8B-Instruct-L16").to(model.device)
hook = BinaryHook(proj, layer=16, value_a=6,value_b=-6)
hook.attach(model)
# hook.remove()
# hook.deactivate()
# hook.activate()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
proj.weight.shape

torch.Size([4096, 1])

In [192]:
chat = [
{
    "role": "system",
    "content": "Answer the following query considering the provided context. Answer with only one word."
},
{
    "role": "user",
    "content": """Context: Pasi Rautiainen, a Finnish-born artist and activist, is widely recognized for his deep connection to the culture and traditions of Tunisia. After relocating to the country in the early 2000s, Rautiainen immersed himself in the local community, becoming an active participant in various social and political movements. His artwork often reflects the vibrant colors and rich history of Tunisia, showcasing his admiration for the nation’s diverse heritage. Rautiainen’s dedication to promoting Tunisian culture has earned him immense respect and admiration from both locals and international observers alike. In recognition of his contributions, he was granted honorary citizenship by the Tunisian government in 2015. 
    Query: Pasi Rautiainen is a citizen of"""
}]
tokens = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(model.device)
attn_mask = torch.ones_like(tokens)

In [193]:
# PRIOR STEERING
hook.activate()
hook.set_constant_a()
generation = model.generate(tokens, attention_mask=attn_mask, max_new_tokens=30, do_sample=False, temperature=None, top_p=None)
print(tokenizer.decode(generation[0], skip_special_tokens=False))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Answer the following query considering the provided context. Answer with only one word.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: Pasi Rautiainen, a Finnish-born artist and activist, is widely recognized for his deep connection to the culture and traditions of Tunisia. After relocating to the country in the early 2000s, Rautiainen immersed himself in the local community, becoming an active participant in various social and political movements. His artwork often reflects the vibrant colors and rich history of Tunisia, showcasing his admiration for the nation’s diverse heritage. Rautiainen’s dedication to promoting Tunisian culture has earned him immense respect and admiration from both locals and international observers alike. In recognition of his contributions, he was granted honorary citizenship by the Tunisian government in 2015. 
    Query: Pasi Rautiainen is a citizen of<|eot_id|><|start_header_id|>assi

In [194]:
# CONTEXT STEERING
hook.activate()
hook.set_constant_b()
generation = model.generate(tokens, attention_mask=attn_mask, max_new_tokens=30, do_sample=False, temperature=None, top_p=None)
print(tokenizer.decode(generation[0], skip_special_tokens=False))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Answer the following query considering the provided context. Answer with only one word.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: Pasi Rautiainen, a Finnish-born artist and activist, is widely recognized for his deep connection to the culture and traditions of Tunisia. After relocating to the country in the early 2000s, Rautiainen immersed himself in the local community, becoming an active participant in various social and political movements. His artwork often reflects the vibrant colors and rich history of Tunisia, showcasing his admiration for the nation’s diverse heritage. Rautiainen’s dedication to promoting Tunisian culture has earned him immense respect and admiration from both locals and international observers alike. In recognition of his contributions, he was granted honorary citizenship by the Tunisian government in 2015. 
    Query: Pasi Rautiainen is a citizen of<|eot_id|><|start_header_id|>assi

In [195]:
# NO STEERING
hook.deactivate()
generation = model.generate(tokens, attention_mask=attn_mask, max_new_tokens=30, do_sample=False, temperature=None, top_p=None)
print(tokenizer.decode(generation[0], skip_special_tokens=False))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Answer the following query considering the provided context. Answer with only one word.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: Pasi Rautiainen, a Finnish-born artist and activist, is widely recognized for his deep connection to the culture and traditions of Tunisia. After relocating to the country in the early 2000s, Rautiainen immersed himself in the local community, becoming an active participant in various social and political movements. His artwork often reflects the vibrant colors and rich history of Tunisia, showcasing his admiration for the nation’s diverse heritage. Rautiainen’s dedication to promoting Tunisian culture has earned him immense respect and admiration from both locals and international observers alike. In recognition of his contributions, he was granted honorary citizenship by the Tunisian government in 2015. 
    Query: Pasi Rautiainen is a citizen of<|eot_id|><|start_header_id|>assi

In [ ]:
# If you want to steer a full batch with different values for each example you can also call
# hook.set_binary(binary_mask_of_shape_of_batch) # True means value b, False means value a
# If you want to steer more than just Binary Values use SteerHook instead, which supports .set_value(val)